<a href="https://colab.research.google.com/github/JeJaMel/AI_Facial_Expressions_Recognicion/blob/main/Facial_expressions_recognicion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyngrok streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 118.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 123.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.7 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
tf.test.gpu_device_name() #this line is for test if T4 GPUT is beeing used (recomend)
import kagglehub
from google.colab import drive
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras import layers
import pandas as pd
import cv2
from tensorflow.keras import mixed_precision
from tensorflow.keras.callbacks import ModelCheckpoint
import streamlit as st
from pyngrok import ngrok
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.models import load_model

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("thienkhonghoc/affectnet")

print("Path to dataset files:", path)

100%|██████████| 1.75G/1.75G [01:26<00:00, 21.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/thienkhonghoc/affectnet/versions/3


In [ ]:
!cp -r /root/.cache/kagglehub/datasets/thienkhonghoc/affectnet/versions/3 /content/drive/MyDrive/affectnet #Move the dataset to the drive

In [ ]:
#Dataset route
DATASET_PATH = '/content/drive/MyDrive/affectnet/AffectNet/train'

In [ ]:
image_size = (224, 224)
batch_size = 32

train_ds = keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.3,  #40% of the images will make part of the validation ds
    subset="training",
    seed=1330,
    image_size=image_size,
    batch_size=batch_size,
)

# Obtain the validation subset
temp_val_ds = keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.3,  # 40%
    subset="validation",
    seed=1330,
    image_size=image_size,
    batch_size=batch_size,
)

val_size = int(0.5 * len(temp_val_ds))
val_size
val_ds = temp_val_ds.take(val_size)
test_ds = temp_val_ds.skip(val_size)

Found 37553 files belonging to 8 classes.
Using 26288 files for training.
Found 37553 files belonging to 8 classes.
Using 11265 files for validation.


In [ ]:
def make_mobilenetv2_model(input_shape, num_classes):
    base_model = keras.applications.MobileNetV2(
        input_shape=input_shape,
        include_top=False,  # Exclude the original classifier
        weights="imagenet"  # Use pre-trained weights
    )

    # Freeze the base model for transfer learning
    base_model.trainable = False

    inputs = keras.Input(shape=input_shape)

    # Normalize input
    x = layers.Rescaling(1.0 / 255)(inputs)

    # Use MobileNetV2 as feature extractor
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)  # Reduce feature maps to a vector

    # Add dropout for regularization
    x = layers.Dropout(0.5)(x)

    # Output layer
    activation = "softmax" if num_classes > 2 else "sigmoid"
    outputs = layers.Dense(num_classes, activation=activation)(x)

    return keras.Model(inputs, outputs)

    mobilenetv2.summary()

In [ ]:
mobilenetv2 = make_mobilenetv2_model(input_shape=(224, 224, 3), num_classes=8)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
mobilenetv2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),  # Lower LR for transfer learning
    loss="sparse_categorical_crossentropy",  # For multi-class classification (6 classes)
    metrics=["accuracy"]  # Track accuracy
)

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True)
]


In [ ]:
history = mobilenetv2.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25,  # Start with 20, adjust based on results
    callbacks=callbacks
)

Epoch 1/25
822/822 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - accuracy: 0.1918 - loss: 2.2961

822/822 ━━━━━━━━━━━━━━━━━━━━ 221s 254ms/step - accuracy: 0.1918 - loss: 2.2959 - val_accuracy: 0.2766 - val_loss: 1.9076
Epoch 2/25
822/822 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - accuracy: 0.2668 - loss: 1.9539

822/822 ━━━━━━━━━━━━━━━━━━━━ 211s 201ms/step - accuracy: 0.2668 - loss: 1.9539 - val_accuracy: 0.3098 - val_loss: 1.8204
Epoch 3/25
822/822 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.2851 - loss: 1.9184

822/822 ━━━━━━━━━━━━━━━━━━━━ 183s 178ms/step - accuracy: 0.2851 - loss: 1.9184 - val_accuracy: 0.3120 - val_loss: 1.8114
Epoch 4/25
822/822 ━━━━━━━━━━━━━━━━━━━━ 201s 177ms/step - accuracy: 0.2851 - loss: 1.9070 - val_accuracy: 0.3058 - val_loss: 1.8181
Epoch 5/25
821/822 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.2866 - loss: 1.9007

822/822 ━━━━━━━━━━━━━━━━━━━━ 200s 175ms/step - accuracy: 0.2866 - loss: 1.9007 - val_accuracy: 0.3137 - val_loss: 1.8084
Epoch 6/25
822/822 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.2913 - loss: 1.9055

822/822 ━━━━━━━━━━━━━━━━━━━━ 201s 174ms/step - accuracy: 0.2913 - loss: 1.9055 - val_accuracy: 0.3246 - val_loss: 1.7984
Epoch 7/25
821/822 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - accuracy: 0.2910 - loss: 1.9049

822/822 ━━━━━━━━━━━━━━━━━━━━ 206s 179ms/step - accuracy: 0.2910 - loss: 1.9049 - val_accuracy: 0.3288 - val_loss: 1.7912
Epoch 8/25
822/822 ━━━━━━━━━━━━━━━━━━━━ 215s 195ms/step - accuracy: 0.2910 - loss: 1.9028 - val_accuracy: 0.3255 - val_loss: 1.7921
Epoch 9/25
822/822 ━━━━━━━━━━━━━━━━━━━━ 143s 174ms/step - accuracy: 0.2938 - loss: 1.8969 - val_accuracy: 0.3113 - val_loss: 1.8095
Epoch 10/25
822/822 ━━━━━━━━━━━━━━━━━━━━ 204s 177ms/step - accuracy: 0.2902 - loss: 1.9005 - val_accuracy: 0.3159 - val_loss: 1.8254
Epoch 11/25
822/822 ━━━━━━━━━━━━━━━━━━━━ 150s 183ms/step - accuracy: 0.2839 - loss: 1.9137 - val_accuracy: 0.3223 - val_loss: 1.8022
Epoch 12/25
822/822 ━━━━━━━━━━━━━━━━━━━━ 196s 176ms/step - accuracy: 0.2852 - loss: 1.9084 - val_accuracy: 0.3217 - val_loss: 1.7996


In [ ]:
pd.DataFrame(history.history).plot(figsize=(10, 7))
plt.grid(True)
plt.gca().set_ylim(0, 1.2)
plt.xlabel("epochs")
plt.show()

In [ ]:
# Save the entire model (architecture, weights, optimizer, etc.)
mobilenetv2.save('/content/drive/MyDrive/trashnet/trainedTrashNetV11(not the best).h5')

In [ ]:
# We test the model with test_ds, images that mobilenetv2 never seen befote
evaluation_result = mobilenetv2.evaluate(test_ds)

print("Loss:", evaluation_result[0])
print("Accuracy:", evaluation_result[1])